In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.datasets as datasets

import theano
import theano.tensor as T
from theano import function,grad,pp,shared

In [2]:
batch_size = 30
ds = datasets.load_iris()
train_data_x = ds['data'][:,0:2]
train_data_y = ds['target']

# plt.plot(x[y==0][:,0],x[y==0][:,1],'+r')
# plt.plot(x[y==1][:,0],x[y==1][:,1],'+b')
# plt.plot(x[y==2][:,0],x[y==2][:,1],'ob')

# i = T.dscalar('i')
# x = T.dmatrix('x')
# y = T.ivector('y')
# W = shared(value=np.random.uniform(low=-0.1,high=0.1,size=(2,1)))

# z = T.dot(x,W) + y.reshape((150,1)) + i 

# f = function([i],z,givens=[[x,train_data_x],[y,train_data_y]])
# f(10)

In [5]:
class LogisticRegression(object):
    def __init__(self,n_in,n_out):
        self.x = T.matrix('x')
        self.y = T.ivector('y')
        self.W = shared(
            value=np.random.uniform(low=-0.1,high=0.1,size=(n_in,n_out)),
            name='W',
            borrow=True
        )
        self.b = shared(
            value=np.random.uniform(low=-0.1,high=0.1,size=(n_out,)),
            name='b',
            borrow=True
        )
        self.cost = T.mean((self.y.reshape((batch_size,1)) - T.tanh(T.dot(self.x ,self.W) + self.b)) ** 2)
        self.output = None
    
train_set_x = shared(value=train_data_x,name='train_set_x',borrow=True)
train_set_y = T.cast(shared(value=train_data_y,name='train_set_y',borrow=True),'int32')

learning_rate = 0.01
index = T.iscalar('index')
model = LogisticRegression(2,1)
g_w = grad(model.cost,model.W)
g_b = grad(model.cost,model.b)

train = function(inputs=[index],outputs=model.cost,
         updates=[
                 (model.W, model.W - learning_rate * g_w),
                 (model.b, model.b - learning_rate * g_b)
                 ],
         givens=[(model.x,train_set_x[index * batch_size :(index+1) * batch_size ]),
                 (model.y,train_set_y[index * batch_size :(index+1) * batch_size ])])



In [6]:
print train(0)
print train(1)
print train(2)

0.0312188022015
0.314632288372
0.439194581232
